# Quantum Scattering

## Introduction

This iPython demo will start to give you some intuition for quantum scattering through a time dependent picture. 

To do this we will implement the Crank-Nicholson scheme, which provides an incredibly efficient and accurate technique for propagating wavefunctions forward in an implicit fashion. The technique works by assuming that:

$$\psi(t + dt) = \frac{1 - \frac{1}{2}i H dt }{1 + \frac{1}{2}i H dt} \psi(t) $$

Now, it's quite difficult to propagate this equation as written, but we find that it's relatively easy to write in an implicit fashion. This means that we're solving a system of equations where:

$$ (1 + \frac{1}{2}i H dt) \psi(t + dt) = (1 + \frac{1}{2}i H dt)^\dagger \psi(t ).$$
$$ A \psi(t + dt) = A^\dagger \psi(t ).$$


This can be thought of as the average of the forward propagator applied at time $t$ and the reverse propagator applied at time $t + dt.$ Then, this scheme basically works to ensure that quantum mechanics maintains its time-reversal symmetry at all times. This is precisely the bit that makes the Crank-Nicholson propagator very robust.

For the purpose of this demo we just want you get familiar with the steps of the propagation algorithm, as you will need it for the next homework. We also want you to play around with the initial conditions of propagation (momentum of the initial wavepacket and height of the barrier) and see how these conditions impact the scattering process.<br><br>


## <i class="fa fa-book"></i> Step 1: Import the required modules and define common variables.

Take note of the variables we are defining as you will have to play around with them. 

In [ ]:
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg
from scipy.integrate import simps
import imp
qworld = imp.load_source('qworld', '../library/quantumWorld.py')

% pylab inline

m = 938.27        # mass
hbar = 0.1973        # hbar
c = 3.0e2         # speed of light

dx = 0.001 # Distance between x points
a = -1.2
b = 1.2
x = np.arange(a, b, dx)
nx = len(x)
pi = np.pi
print np.shape(x)

T = 0.07 # Total time

dt = .1e-3 # dt
time_steps = int(T/dt) # Number of time steps
niter = 1 # Save psi_t every niter steps
time = np.arange(0, T, dt)

## <i class="fa fa-book"></i> Step 2: Let's define some useful functions for the propagation.

Get familiar with the names of the following functions, as you will be calling them in the next steps.

In [ ]:
##############################
# BLACK BOX METHODS
##############################

def Psi0(x, k=200000.0, x0 = -0.100):
    '''
    Initial state for a travelling gaussian wave packet.
    '''
    a  = 0.030 
    prefactor = (1.0/(2.0*pi*a**2))**0.25
    K1 = np.exp(-1.0*(x-x0)**2/(4.*a**2))
    K2 = np.exp(1j*k*x)
    return prefactor*K1*K2

def delta_abs(x, height = 5, cutoff = 0.4):
    '''
    This function takes in an array of positions and returns a scattering potential
    with absorbing boundary conditions.
    Just use as a black box
    '''
    potential = np.zeros(len(x), dtype=complex)
    potential[0.5*len(potential)] = height
    d = 0.1
    for i in range(len(x)):
        if x[i] >= cutoff:
            potential[i] = -1j*(1.0/(np.cosh((x[i] - cutoff)**2/d**2))**2 - 1.0)
        if x[i] <= -cutoff:
            potential[i] = -1j*(1.0/(np.cosh((x[i] + cutoff)**2/d**2))**2 - 1.0)
    return potential

def build_KE(dx, nx):
    '''
    THis method just returns the tridiagonal kinetic energy. Ask if you have any questions.
    dx: separation in the x grid
    nx: number of points in the x grid
    '''
    prefactor = -(1j*hbar*c)/(2.*m)
    data = np.ones((3, nx))
    data[1] = -2*data[1]
    diags = [-1,0,1]
    D2 = prefactor / dx**2 * sparse.spdiags(data,diags,nx,nx)
    return D2


def build_potential(x, V_func, nx):
    '''
    This method just returns a sparse diagonal matrix with the potential on the diagonals
    x: x grid
    V_func: function that defines the potential (we use delta_abs in this example)
    nx: number of points in the x grid
    '''
    k2 = (1j*c)/hbar
    V_data = V_func(x)
    V_diags = [0]
    V = k2 * sparse.spdiags(V_data, V_diags, nx, nx)
    I = sparse.identity(nx)
    return V, V_data, I

def forwardcn(psi, A, Ad):
    '''
    This method takes one step forward using the crank nicholson propagator. As promised, it uses the sparse solver
    to find where A \psi(t + dt) = A^\dagger \psi(t)
    '''
    psi = sparse.linalg.spsolve(A,Ad*psi)
    return psi


def animate_psi(psit, name='test.mp4'):
    '''
    This method produces an animation of the saved wavefunctions. Note, this can be a slow method
    '''
    def init():
        '''
        Animation interior function
        '''
        line.set_data([], [])
        return line,

    def animate(i):
        '''
        Animation interior function
        '''
        line.set_data(x, np.abs(psit[i,:]/2.0)**2)
        return line,
    
    from matplotlib import animation
    fig = plt.figure()
    fig.set_size_inches(10, 8)
    ax = plt.axes(xlim=(-0.5, 0.5), ylim=(-0.1, 10.0)) #create single axis in figure
    ax.plot(x, V_data/5.0, lw=2)
    line, = ax.plot([], [], lw=2)
    ani = animation.FuncAnimation(fig, animate, np.arange(1, np.shape(psit)[0]), init_func=init)
    ani.save(name, fps=60)
    return ani

## <i class="fa fa-wrench"></i> Step 3: Now create a wavepacket to propagate. 

First, we need to create a wavepacket. Call the Psi0 method that is defined above. Note that k is the incoming momentum of the packet. For this first test, lets use k=-200.0. (The minus sign is related with the direction of the momentum. This wavepacket will move to the right.)

In [ ]:
# Define the value of the momentum for your wavepacket
k_value=-100

# CALL THE FUNCTION Psi0 TO BUILD A WAVEPACKET WITH k=k_value AND INITIAL POSITION (x0) of -0.2
# assign it to the variable PsiaR (1 LINE)
PsiaR = 
# normalizing the wavepacket
PsiaR = PsiaR/np.sqrt(simps(np.abs(PsiaR)**2, x))

# verifying normalization
print simps(np.abs(PsiaR)**2, x)

We will now generate the matrices for the potential energy and the kinetic energy, that we will use to build our Hamiltonian. We will also create an array to save the wavepacket wavefunctions as a function of time and the autocorrelation function.

In [ ]:
# CALL THE FUNCTION build_KE THAT CONSTRUCT THE KINETIC ENERGY MATRIX (1 LINE)
# assign it to the variable D2
D2 = 
#build a delta function barrier (use the "black_box" definition delta_abs)
# by calling THE FUNCTION build_potential THAT CONSTRUCT THE POTENTIAL ENERGY MATRIX (1 LINE)
# assign the output to the variables V, V_data and I
V, V_data, I = 

# Generating an array to store the wavepacket at different times
psitaR = np.zeros((int((time_steps)/niter+1), len(PsiaR)), dtype=complex)
# generating a matrix to store the correlation function at different times
Caa = np.zeros(( int(time_steps/niter+1), ), dtype=complex)

Now assign to the first column of psitaR (psitaR[0, :]) the variable PsiaR, that corresponds to the initial wavepacket. Also compute the first element of the autocorrelation function. The autocorrelation function at a time $t'$ for the incident wavepacket is defined as:

$$ C(t')=\int \psi(t=t',x)^* \psi(t=0,x)  dx $$

In [ ]:
# ASSIGN PsiaR TO THE FIRST ELEMENT OF THE VARIABLES psitaR (1 LINe)
psitaR[0, :] = 

# COMPUTE THE AUTOCORRELATION FUNCTION AT t=0 (1 LINE)  
Caa[0] = 

## <i class="fa fa-wrench"></i> Step 4: Perform the propagation

The first thing we need to do before propagating is computing the $A$ (A) and $A^\dagger$ (Ad) Matrices required for the Crank-Nicholson propagation. According to the introduction these matrices will be given by:
$$A = I - \frac{dt}{2}(D2 + V) $$
and
$$A^\dagger = I + \frac{dt}{2}(D2+V)$$
(Note that D2 already has the imaginary prefactor)

In [ ]:
# DEFINE THE A AND A^{dagger} MATRICES (2 LINES)
# Remember we have defined the kinetic energy and potential energy matrix in the previous step as D2 and V.
A = 
Ad = 

Now fill in the loop such that it updates the wavefunction *Psiar* using the *forwardcn* method and fill in the correlation function computation.

In [ ]:
j = 1
for i in range(time_steps):
    # CALL THE forwardcn FUNCTION TO PROPAGATE 1 TIME STEP, ASSIGN THE RESULT TO PsiaR (1 LINE)
    PsiaR = 
    if i%niter == 0:
        # SAVE THE NEW PsiaR VALUE TO THE j-th ELEMENT OF THE psitaR MATRIX (1 LINE)
        psitaR[j, :] = 
        # COMPUTE THE AUTOCORRELATION FUNCTION FOR THE j-th TIME STEP (1 LINE)  
        Caa[j] = 
        j+=1

Now we will plot the value of the autocorrelation function as a function of time.

In [ ]:
tarray=dt*np.arange(0,len(Caa))
plt.figure(1)
plt.ylabel('|C(t)|',fontsize=12)
plt.xlabel('Time',fontsize=12)
plt.title("Autocorrelation function: $<\psi(0) | \psi(t)>$")
plt.plot(tarray,np.abs(Caa))
plt.show()

## <i class="fa fa-book"></i> Step 6: Animate the propagation.

In [ ]:
anim=animate_psi(psitaR)
qworld.embedAnimation(anim,plt)

## <i class="fa fa-wrench"></i> Step 7: Time to play around.

Now we want you to play around with the momentum and the shape of the potential.

1. Now, try a smaller k (Let's say k=-800). What happens with the transmission? Note that k is related to the temperature, so a larger k corresponds to a higher temperature and a smaller k to a lower temperature.
1. Change the height of the potential in the definition of the function delta_abs. The default value is 5, raise it to 10. How does that change the transmission?